In [6]:
from typing import Dict, TypedDict
from langgraph.graph import StateGraph
from IPython.display import Image, display

In [4]:
class AgentState(TypedDict):
    """
    Our state schema
    """
    message: str

def greeting_node(state:AgentState) -> AgentState:
    """
    Simple node that adds greeting message to the state
    """
    state["message"] = "Hey "+ state["message"] + " how is your day going?"
    return state

In [5]:
graph = StateGraph(AgentState)

graph.add_node("greeter", greeting_node)
graph.set_entry_point("greeter")
graph.set_finish_point("greeter")
app = graph.compile()

In [ ]:
display(Image(app.get_graph()